In [1]:
import pandas as pd
import os
import chardet

In [17]:
letter = 'w'
df_new = pd.read_csv('./df.csv')
with open(f'./data/{letter}.csv', 'rb') as f:
    result = chardet.detect(f.read())
    encoding = result['encoding']
letter_csv = pd.read_csv(f'./data/{letter}.csv', encoding=encoding)


In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
import requests
import os
import csv

In [3]:
def collect_font_data(link, browser):
    font_info = []
    base_link = f'https://www.fontspace.com{link}'

    # Render font page
    browser.get(base_link)
    html_content = browser.page_source
    soup = BeautifulSoup(html_content, 'html.parser')

    # Collect data
    # {name: img: , tags: []}
    try:
        section = soup.find('section')
        name = section.find('h1').text[:-5]

        img = "N/A"
        hero_img = soup.find('img', {'alt': f'{name} Font'})
        # regular_img = soup.find('img', {'alt': f'{name[:-5]}'})
        first_lazy_img = soup.find('img', {'class': 'v-lazy-image', 'data-ratio': '1'})
        if hero_img:
            img = hero_img['src']
        elif first_lazy_img:
            img = first_lazy_img['src']

        tag_list = soup.find('ul', {'class': 'topic-list'})
        tag_list_items = soup.find_all('li', {'class': 'topic-btn'})
        tags = []
        for tag in tag_list_items:
            tags.append(tag.find('h4').text)

        return {
            'name': name,
            'img': img,
            'tags': tags
        }
    except AttributeError:
        print(f'? Failed collecting data for ${base_link}')
    return

In [4]:
def render_letter_fonts(url, browser):
    info = []

    browser.get(url)

    # Render page
    html_content = browser.page_source
    soup = BeautifulSoup(html_content, 'html.parser')

    # Grab font list
    links = soup.find_all('a', {'class': 'font-image'})

    # Grab font data from list
    for link in links:
        info.append(collect_font_data(link['href'], browser))

    # Return 
    return info

In [9]:
fontspace_link = 'https://www.fontspace.com/list/'
letter_length_dict = {'a': 200, 'b': 254, 'c': 207, 'd': 139, 'e': 68, 'f': 110, 'g': 111, 'h': 135, 'i': 34, 'j': 62, 'k': 108, 'l': 111, 'm': 205, 'n': 59, 'o': 42, 'p': 112, 'q': 23, 'r': 137, 's': 284, 't': 121, 'u': 19, 'v': 62, 'w': 69, 'x': 8, 'y': 19, 'z': 20, 'other': 20}

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.binary_location = '/Applications/Brave Browser.app/Contents/MacOS/Brave Browser'
browser = webdriver.Chrome(options=options)

for letter in tqdm(letter_length_dict, desc=f'Collection Progress'):
    letter_data = []
    letter_length = letter_length_dict[letter]
    inner_progress = tqdm(range(1, letter_length), desc=letter, leave=True)
    for x in inner_progress:
        url = f"{fontspace_link}{letter}?p={x}"
        letter_data.extend(render_letter_fonts(url, browser))
    with open(f"data/new_{letter}.csv", 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['name', 'img', 'tags'])
        writer.writeheader()
        for row in letter_data:
            try:
                writer.writerow(row)
            except AttributeError:
                print(f'Failed writing a row to new_{letter}.csv')
    inner_progress.reset()
browser.quit()

Collection Progress:   0%|          | 0/27 [00:00<?, ?it/s]

? Failed collecting data for $https://www.fontspace.com/ashleigh-patty-font-f82080


Collection Progress:   4%|▎         | 1/27 [53:50<23:19:56, 3230.63s/it]

Failed writing a row to new_a.csv


Collection Progress:   4%|▎         | 1/27 [57:20<24:50:48, 3440.33s/it]


KeyboardInterrupt: 